# Рекоммендательные системы
### Дуботолкова Натали, 3530903/70301
### Вариант 7

In [197]:
import numpy as np
import pandas as pd
import json
import math as m
%matplotlib inline

In [157]:
data = pd.read_csv('data.csv', index_col=0)

In [158]:
data.head(7)

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,4,1,5,-1,-1,-1,1,3,-1,...,2,4,-1,-1,3,4,-1,2,4,1
User 2,4,5,2,3,3,4,-1,5,3,2,...,-1,2,-1,3,1,5,-1,5,2,2
User 3,4,2,5,4,1,5,-1,3,5,2,...,4,-1,5,4,5,1,2,1,-1,3
User 4,5,-1,2,-1,-1,2,4,3,2,3,...,-1,2,2,2,2,5,3,2,5,5
User 5,4,3,3,3,5,3,3,4,5,2,...,4,5,2,-1,3,5,1,-1,5,-1
User 6,2,-1,3,3,3,3,4,2,-1,1,...,5,-1,1,3,-1,4,4,5,1,2
User 7,4,1,4,4,1,4,5,2,2,-1,...,2,5,1,3,4,-1,4,3,1,4


Для заданного пользователя (совпадает с вашим номером в таблице) рассчитать оценки для всех фильмов, которые он не оценил.

In [159]:
data.iloc[2][1]

2

In [160]:
k = 4
my_user = 6

In [161]:
def sim(u,v):
    sum_uv, sum_u, sum_v = 0, 0, 0
    for i in range (len(u)):
        if u[i]>0:
            sum_u += u[i]**2
            if v[i]>0:
                sum_v += v[i]**2
                sum_uv += v[i]*u[i]
    return round((sum_uv/(m.sqrt(sum_u)*m.sqrt(sum_v))),3)
        

In [162]:
all_similarities = np.zeros((40, 40))

In [163]:
for i in range(data.shape[0]):
    listOfU = list(data.iloc[i])
    j = i+1
    for j in range(j, data.shape[0]):
        listOfV = list(data.iloc[j])
        all_similarities[i, j] = sim(listOfU, listOfV)
        all_similarities[j, i] = all_similarities[i, j]

In [164]:
all_similarities #матрица сходств всех пользователей

array([[0.   , 0.736, 0.63 , ..., 0.755, 0.635, 0.813],
       [0.736, 0.   , 0.7  , ..., 0.719, 0.628, 0.777],
       [0.63 , 0.7  , 0.   , ..., 0.762, 0.756, 0.627],
       ...,
       [0.755, 0.719, 0.762, ..., 0.   , 0.749, 0.806],
       [0.635, 0.628, 0.756, ..., 0.749, 0.   , 0.783],
       [0.813, 0.777, 0.627, ..., 0.806, 0.783, 0.   ]])

In [165]:
my_similarities = all_similarities[my_user] #матрица сходств для моего пользователя (7-го)

In [166]:
my_similarities

array([0.721, 0.778, 0.815, 0.736, 0.77 , 0.789, 0.   , 0.751, 0.714,
       0.724, 0.661, 0.786, 0.799, 0.734, 0.658, 0.839, 0.642, 0.811,
       0.783, 0.729, 0.837, 0.674, 0.69 , 0.772, 0.71 , 0.739, 0.759,
       0.76 , 0.763, 0.874, 0.818, 0.84 , 0.779, 0.734, 0.736, 0.791,
       0.694, 0.828, 0.738, 0.801])

In [167]:
my_LikeMinded = my_similarities.argsort()[-k:]
my_LikeMinded #массив индексов четырёх людей, у кого со мной наибольшее сходство

array([20, 15, 31, 29], dtype=int32)

In [174]:
#средняя оценка пользователя u
def average(u):
    sum_u = 0
    counter = 0
    for i in range (len(u)):
        if u[i]>0:
            counter += 1
            sum_u += u[i]
    return round((sum_u/counter),3)

In [175]:
# расчёт оценки на конкретный фильм
def r_ui(film):
    listOfU = list(data.iloc[my_user])
    r_u = average(listOfU)
    sum_up, sum_down = 0, 0
    for i in range(k):
        v = my_LikeMinded[i]
        listOfV = list(data.iloc[v])
        _sim = sim(listOfU, listOfV)
        sum_up += round((_sim*(data.iloc[v][film]-average(listOfV))), 3)
        sum_down += abs(_sim)
    return round((r_u + sum_up/sum_down),3)

In [201]:
listOfU = list(data.iloc[my_user])
listRes = {}
# находим индексы непросматриваемых фильмов и пытаемся предсказать оценку
for idx, num in enumerate(listOfU):
        if num<0:
            listRes['movie ' + str(idx+1) + ': '] = r_ui(idx)
listRes

{'movie 10: ': 1.539,
 'movie 12: ': 1.549,
 'movie 17: ': 1.799,
 'movie 18: ': 2.069,
 'movie 26: ': 3.807}

In [202]:
result = {"user": 7, "1": listRes}
with open('result.json', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)